---

### **1. Import Module**

In [49]:
import os
import random
import glob
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw

---

In [42]:
# # 이미지가 있는 폴더 경로
# image_folder = './modified_dataset/0/'  # 이미지 폴더 경로를 수정하세요.

# # 이미지 파일 목록 가져오기
# image_files = glob.glob(os.path.join(image_folder, '*.jpg'))

# # 이미지 개수를 받아 그 만큼 Crop, Rotation
# def random_cropRotation_image(img_num):
#     # 5개의 무작위 이미지 선택
#     selected_images = random.sample(image_files, img_num)

#     for image_path in selected_images:
#         # 이미지 불러오기
#         image = Image.open(image_path)

#         # 원 모양의 크롭 영역 생성
#         center_x, center_y = image.width // 2, image.height // 2
#         radius = min(center_x, center_y)  # 반지름을 이미지 크기의 최소 길이의 절반으로 설정
#         mask = Image.new('L', image.size, 0)
#         draw = ImageDraw.Draw(mask)
#         draw.ellipse((center_x - radius, center_y - radius, center_x + radius, center_y + radius), fill=255)
#         cropped_image = Image.composite(image, Image.new('RGB', image.size), mask)

#         # 이미지 회전 (예: 45도)
#         rotated_cropped_image = cropped_image.rotate(45)

#         # 크롭된 이미지 및 회전된 이미지 출력
#         plt.figure(figsize=(8, 4))

#         plt.subplot(1, 2, 1)
#         plt.imshow(cropped_image)
#         plt.title('Cropped Image')

#         plt.subplot(1, 2, 2)
#         plt.imshow(rotated_cropped_image)
#         plt.title('Rotated Cropped Image')

#         plt.show()

---

### **2. Define Crop Function**

In [50]:
# 이미지 폴더 경로 설정
image_folder = './modified_dataset/'

# 이미지 개수를 받아 그 만큼 Crop
def crop_image(query):
    set_image_folder = image_folder + str(query) + '/'

    # 이미지 파일 목록 가져오기
    image_files = glob.glob(os.path.join(set_image_folder, '*.jpg'))

    for image_path in image_files:
        # 이미지 불러오기
        image = Image.open(image_path)

        # 원 모양의 크롭 영역 생성
        center_x, center_y = image.width // 2, image.height // 2
        radius = min(center_x, center_y)  # 반지름을 이미지 크기의 최소 길이의 절반으로 설정
        mask = Image.new('L', image.size, 0)
        draw = ImageDraw.Draw(mask)
        draw.ellipse((center_x - radius, center_y - radius, center_x + radius, center_y + radius), fill=255)
        cropped_image = Image.composite(image, Image.new('RGB', image.size), mask)

        # 저장할 폴더 경로
        save_folder = './cropped_dataset/' + str(query) + '/'  # 저장할 폴더 경로를 수정하세요.

        # 이미지 저장 (파일명은 원본 파일명과 동일)
        save_path = os.path.join(save_folder, os.path.basename(image_path))
        cropped_image.save(save_path)

---

### **3. Define Data Shuffle Function**

In [ ]:
# 이미지 파일 목록 섞기 (랜덤화)
random.shuffle(image_files)

# 이미지 파일 목록을 train과 test로 8:2로 나누기
total_images = len(image_files)
train_ratio = 0.8  # train 비율 설정 (0.8은 80%를 train으로 사용)
train_images = image_files[:int(total_images * train_ratio)]
test_images = image_files[int(total_images * train_ratio):]

In [11]:
print("데이터 비율: " + str(len(train_images) / total_images))

데이터 비율: 0.7980132450331126


---

### **00. Data Classification**

In [48]:
for i in range(3):
    crop_image(i)
    print("cropped_dataset[" + str(i) + "]에 저장 완료.")

cropped_dataset[0]에 저장 완료.
cropped_dataset[1]에 저장 완료.
cropped_dataset[2]에 저장 완료.


---

In [ ]:
# 이미지 회전 (예: 45도)
rotated_cropped_image = cropped_image.rotate(45)

# 크롭된 이미지 및 회전된 이미지 출력
plt.figure(figsize=(8, 4))

plt.subplot(1, 2, 1)
plt.imshow(cropped_image)
plt.title('Cropped Image')

plt.subplot(1, 2, 2)
plt.imshow(rotated_cropped_image)
plt.title('Rotated Cropped Image')

plt.show()

In [ ]:
# # 선택한 이미지들을 처리하고 출력
# for image_path in train_images:

# for image_path in test_images: